In [ ]:
import pandas as pd
master_path = "/content/drive/MyDrive/Applied Machine Learning Project/h-and-m-personalized-fashion-recommendations"
pp_path = "/content/drive/MyDrive/Applied Machine Learning Project/preprocessed_data"
csv_train = pp_path+"/pp_transactions_1_day.csv"
csv_users = pp_path+"/pp_customers.csv"
csv_items = master_path+"/articles.csv"



In [ ]:
# df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])
csv_train_1 = pp_path+"/pp_transactions.csv"
df1 = pd.read_csv(csv_train_1, parse_dates=['t_dat'])
# df1 = df1.drop(columns=["Unnamed: 0.1"])
df1.head()

KeyError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
users_df = pd.read_csv(csv_users)
articles_df = pd.read_csv(csv_items, dtype={'article_id': str})

In [ ]:
# get all users and items 
users = users_df['customer_id'].unique().tolist()
items = articles_df['article_id'].unique().tolist()

In [ ]:
customer_ids = dict(list(enumerate(users)))
article_ids = dict(list(enumerate(items)))
df1['user_id']=df1['customer_id'].map({u: uidx for uidx, u in customer_ids.items()})
df1['item_id'] = df1['article_id'].map({int(i): iidx for iidx, i in article_ids.items()})

In [ ]:
# df1.to_csv("/content/drive/MyDrive/Applied Machine Learning Project/preprocessed_data/train_data_full.csv")

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Applied Machine Learning Project/preprocessed_data/train_data_full.csv")
df1.drop(columns=["Unnamed: 0"], inplace=True)
df1.head()

,Unnamed: 0.1,t_dat,customer_id,article_id,price,sales_channel_id,user_id,item_id
0,0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,2,40179
1,1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,2,10520
2,2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,7,6387
3,3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,7,46304
4,4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,7,46305


In [ ]:
dfh = df1.groupby("user_id")['item_id'].apply(lambda items: list(set(items)))
dfh = dfh[dfh.str.len() >= 3]

In [ ]:
df_for_rec = df1.groupby(['user_id', 'article_id']).size().reset_index(name='total_item')
# all_transaction_per_member_df = df_for_rec.groupby('customer_id')['article_id'].apply(list)
# all_transaction_per_member_df = all_transaction_per_member_df.reset_index()
# all_transaction_per_member_df = all_transaction_per_member_df[(all_transaction_per_member_df['article_id'].str.len() > 2) & (all_transaction_per_member_df['article_id'].str.len() < 150)]
# df_for_rec = df_for_rec[df_for_rec['customer_id'].isin(all_transaction_per_member_df.customer_id)]
# df_for_rec = df_for_rec[df_for_rec['total_item'] < 3]
df_for_rec

,user_id,article_id,price,total_item
0,0,176209023,0.035576,1
1,0,568601006,0.050831,2
2,0,568601043,0.050831,1
3,0,607642008,0.012695,1
4,0,625548001,0.044051,1
...,...,...,...,...
28109844,1371978,840360003,0.013542,1
28109845,1371978,866755002,0.043203,1
28109846,1371978,866755002,0.050831,1
28109847,1371978,882810001,0.016932,1


In [ ]:
df_for_rec["user_id"] = df_for_rec["user_id"].astype("category")
df_for_rec["article_id"] = df_for_rec["article_id"].astype("category")
df_for_rec["user_ID"] = df_for_rec["user_id"].cat.codes
df_for_rec["article_ID"] = df_for_rec["article_id"].cat.codes

df_for_rec.to_csv("/content/drive/MyDrive/Applied Machine Learning Project/preprocessed_data/train_df_full.csv")

In [ ]:
import scipy.sparse as sparse
sparse_user_items = sparse.csr_matrix((df_for_rec['total_item'].astype(float), 
                                       (df_for_rec['user_ID'], df_for_rec['article_ID'])))
# sparse_user_items
sparse_items_user = sparse.csr_matrix((df_for_rec['total_item'].astype(float), 
                                       ( df_for_rec['article_ID'],df_for_rec['user_ID'])))

In [ ]:
!pip install --upgrade implicit

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np
import scipy.sparse as sparse
import pandas as pd
master_path = "/content/drive/MyDrive/Applied Machine Learning Project/h-and-m-personalized-fashion-recommendations"
pp_path = "/content/drive/MyDrive/Applied Machine Learning Project/preprocessed_data"
csv_train = pp_path+"/pp_transactions_1_day.csv"
csv_users = pp_path+"/pp_customers.csv"
csv_items = master_path+"/articles.csv"
users_df = pd.read_csv(csv_users)
articles_df = pd.read_csv(csv_items, dtype={'article_id': str})
df_for_rec = pd.read_csv("/content/drive/MyDrive/Applied Machine Learning Project/preprocessed_data/train_df_full.csv")

In [ ]:

class recommender(object):
  def __init__(self, user_df, articles_df, df_for_rec):
    print("Users reading....")
    self.users_df = users_df
    print("Users loaded!")
    print("Articles reading....")

    self.articles_df = articles_df
    print("Articles loaded!")
    print("Training data reading...")
    self.df_for_rec = df_for_rec 
    print("Training data loaded!")
    self.df_for_rec["user_id"] = self.df_for_rec["user_id"].astype("category")
    self.df_for_rec["article_id"] = self.df_for_rec["article_id"].astype("category")
    self.df_for_rec["user_ID"] = self.df_for_rec["user_id"].cat.codes
    self.df_for_rec["article_ID"] = self.df_for_rec["article_id"].cat.codes
    print("Creating Sparse user-items matrix")
    self.sparse_user_items = sparse.csr_matrix((self.df_for_rec['total_item'].astype(float), 
                                       (self.df_for_rec['user_ID'], self.df_for_rec['article_ID'])))
    print("Done!")
    print("Creating Sparse items-user matrix")

    self.sparse_items_user = sparse.csr_matrix((self.df_for_rec['total_item'].astype(float), 
                                          ( self.df_for_rec['article_ID'],self.df_for_rec['user_ID'])))
    print("Done!")
    self.person_vecs = None
    self.content_vecs = None
    self.content_norms = None
    #article has 12 feature, customer estimatley has 6 and transaction data has 5
    os.environ['OPENBLAS_NUM_THREADS']='1'
    os.environ['KMP_DUPLICATE_LIB_OK']='True'
    os.environ['MKL_NUM_THREADS']='1'
    self.als_model = AlternatingLeastSquares(factors=360, regularization=0.01, use_gpu=True, iterations=5, dtype=np.float32, calculate_training_loss=True)
  def train_user_items(self):
    data = (self.sparse_user_items *15).astype("double")
    print("Training model")
    self.als_model.fit(data)
    print("Model trained!")
    self.person_vecs = self.als_model.user_factors.to_numpy()
    self.content_vecs = self.als_model.item_factors.to_numpy()
    self.content_norms = np.sqrt((self.content_vecs * self.content_vecs).sum(axis=1))
  def get_article_info(self, top_idx):
    article_ids = self.df_for_rec[self.df_for_rec.article_ID.isin(top_idx)]["article_id"].unique()
    return self.articles_df[self.articles_df['article_id'].astype("int64").isin(article_ids)]

  def get_recommendation(self, user_idx, top_n):
    scores = self.content_vecs.dot(self.content_vecs[user_idx]) / self.content_norms
    top_idx = np.argpartition(scores, -top_n)[-top_n:]
    similar = sorted(zip(top_idx, scores[top_idx] / self.content_norms[user_idx]), key=lambda x: -x[1])
    article_info = self.get_article_info(sorted(top_idx))
    recs = pd.DataFrame(similar, columns = ["item_idx", "score"])
    temp = self.get_article_info(sorted(top_idx))
    temp = temp.reset_index().drop(columns=["index"])
    recs[list(temp.columns)] = temp
    return recs

In [ ]:
rec = recommender(users_df, articles_df, df_for_rec)

Users reading....
Users loaded!
Articles reading....
Articles loaded!
Training data reading...
Training data loaded!
Creating Sparse user-items matrix
Done!
Creating Sparse items-user matrix
Done!


In [ ]:
rec.train_user_items()

Training model


  0%|          | 0/5 [00:00<?, ?it/s]

Model trained!


In [ ]:
rec.get_recommendation(1,30)

,item_idx,score,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,1,1.000000,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0,0.817356,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,10100,0.463108,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,47206,0.454890,0509133001,509133,Dingo tee TM,255,T-shirt,Garment Upper body,1010014,Placement print,...,Tops Fancy Jersey,D,Divided,2,Divided,53,Divided Collection,1005,Jersey Fancy,Short-sleeved jersey top with a tie at the front.
4,10098,0.449663,0516786001,516786,BB JERSEY BEANIE BASIC,75,Hat/beanie,Accessories,1010016,Solid,...,Young Boy Big Acc,J,"Children Accessories, Swimwear",4,Baby/Children,45,Kids Outerwear,1019,Accessories,"Hat in a double layer of marled, cotton-blend ..."
5,39851,0.446473,0532224002,532224,Flirty Anita bclet pk,68,Bracelet,Accessories,1010016,Solid,...,Jewellery,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories,Bracelets of metal and braided cord decorated ...
6,19544,0.353920,0538699001,538699,V-neck strap top,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with a V-neck and narrow shoulder s...
7,2,0.353875,0538699007,538699,V-neck strap top,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with a V-neck and narrow shoulder s...
8,29786,0.337444,0552075001,552075,Venom,272,Trousers,Garment Lower body,1010016,Solid,...,Trouser,F,Menswear,3,Menswear,55,Contemporary Street,1009,Trousers,"5-pocket skinny-fit jeans in washed, stretch d..."
9,47205,0.308551,0556539046,556539,Velvet scrunchie,73,Hair string,Accessories,1010016,Solid,...,Hair Accessories,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories,Velour-covered hair elastic.


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# content_vecs


In [ ]:

  
get_recommendation(1, person_vecs, content_vecs, content_norms)

,item_idx,score,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,1,1.000000,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0,0.805940,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,10098,0.509956,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,10100,0.507581,0538699001,538699,V-neck strap top,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with a V-neck and narrow shoulder s...
4,39851,0.443163,0538699007,538699,V-neck strap top,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with a V-neck and narrow shoulder s...
5,47206,0.414897,0584618002,584618,Falsterbo sweater,252,Sweater,Garment Upper body,1010008,Front print,...,Kids Girl Knitwear,H,Children Sizes 92-140,4,Baby/Children,76,Kids Girl,1003,Knitwear,"Jumper in a fine, fluffy knit with an intarsia..."
6,19544,0.394334,0626366001,626366,gaby,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey,A,Ladieswear,1,Ladieswear,6,Womens Casual,1005,Jersey Fancy,Fitted top in soft organic cotton jersey with ...
7,29786,0.384418,0626366003,626366,gaby,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey,A,Ladieswear,1,Ladieswear,6,Womens Casual,1005,Jersey Fancy,Fitted top in soft organic cotton jersey with ...
8,29788,0.319666,0662961003,662961,MARMITE,253,Vest top,Garment Upper body,1010005,Colour blocking,...,Jersey Fancy,F,Menswear,3,Menswear,20,Contemporary Smart,1005,Jersey Fancy,Vest top in soft cotton jersey.
9,2,0.319174,0688463003,688463,Straptop 2-pack,253,Vest top,Garment Upper body,1010016,Solid,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Tops in cotton jersey with narrow shoulder str...


In [ ]:

def get_user_info(idx):
  user_ids = df_for_rec[df_for_rec.user_ID.isin([idx])]["user_id"]
  user_true_ids = df1[df1['user_id'].isin(user_ids)]["customer_id"].unique()
  return users_df[users_df["customer_id"].isin(user_true_ids)]